In [1]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from src.data.data_generator import generate_classpass_data
from src.models.booking_prediction import BookingLikelihoodModel
from src.models.churn_model import ChurnModel
from src.pricing.dynamic_pricing import DynamicPricingEngine
from preprocessing import preprocess_features
from train_model import train_booking_model, train_churn_model
from utils import save_model, load_model

# load data
df = generate_classpass_data(50000)

In [2]:
if "booked" not in df.columns:
    df["booked"] = np.random.choice([0, 1], size = len(df))

if "churned" not in df.columns:
    df["churned"] = np.random.choice([0, 1], size = len(df))

if "class_time" not in df.columns:
    df["class_time"] = pd.date_range(start = "2026-01-01", periods = len(df), freq = "h")

print("Columns:", df.columns)
print("Shape:", df.shape)
print(df.head())

Columns: Index(['persona', 'class_type', 'credits_remaining', 'credit_price',
       'current_streak', 'past_bookings_30d', 'social_score',
       'days_since_last_booking', 'class_time', 'churned', 'booked'],
      dtype='object')
Shape: (50000, 11)
            persona          class_type  credits_remaining  credit_price  \
0   Balanced Burner  Heated Mat Pilates                 46            17   
1      Mindful Muse              Lagree                 36            14   
2  Pilates Princess  Heated Mat Pilates                 20            11   
3      Mindful Muse                HIIT                 17            13   
4      Mindful Muse         Yoga Sculpt                 18             8   

   current_streak  past_bookings_30d  social_score  days_since_last_booking  \
0              68                  3     50.948309                       29   
1              38                  9     75.397346                        9   
2              25                  7     66.411833     

In [3]:
# training and testing split for booking model
df_train, df_test = train_test_split(df, test_size = 0.2, random_state = 42)

In [4]:
# booking likelihood model
booking_model = BookingLikelihoodModel()

# training
booking_model.train(df_train)

# predicting on test set
booking_preds = booking_model.predict(df_test)

# evaluating
booking_auc = roc_auc_score(df_test["booked"], booking_preds)
print("\nBooking Model AUC:", booking_auc)

/Users/obianyanwu/classpass-optimization-engine/venv/lib/python3.12/site-packages/xgboost/training.py:200: UserWarning: [15:42:20] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:782: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Booking Model AUC: 0.4986137617560247

Booking Model AUC: 0.4978462978462978


In [5]:
# churn model
churn_model = ChurnModel()

# training 
churn_model.train(df_train)

# predicting on test data
churn_preds = churn_model.predict(df_test)

# evaluate
churn_auc = roc_auc_score(df_test["churned"], churn_preds)
print("Churn Model AUC:", churn_auc)

Churn AUC: 0.49998424475251785
Churn Model AUC: 0.5085341705263509


In [6]:
# dynamic pricing

pricing_engine = DynamicPricingEngine()

df_test["booking_prob"] = booking_preds
df_test["churn_prob"] = churn_preds

# combine booking and churn probabilities into a single demand score
df_test["demand_score"] = df_test["booking_prob"] * (1 - df_test["churn_prob"])

# simulating fill rate
df_test["fill_rate"] = np.random.rand(len(df_test))

# applying pricing engine
df_test["dynamic_price"] = df_test.apply(lambda row: pricing_engine.adjust_price(row["demand_score"], row["fill_rate"]), axis = 1)

print("\nSample of Dynamic Pricing Output:")
print(df_test[["credit_price", "booking_prob", "churn_prob", "demand_score", "fill_rate", "dynamic_price"]].head())


Sample of Dynamic Pricing Output:
       credit_price  booking_prob  churn_prob  demand_score  fill_rate  \
33553            14      0.441710    0.496582      0.222365   0.504310   
9427             12      0.558018    0.507958      0.274568   0.707826   
199              13      0.458634    0.499955      0.229338   0.615924   
12447             9      0.513727    0.498747      0.257507   0.506002   
39489            19      0.462389    0.502517      0.230030   0.154539   

       dynamic_price  
33553          10.44  
9427           10.55  
199            10.46  
12447          10.52  
39489           8.00  


In [ ]:
edited booking likelihood and churn model training